In [1]:
from pulp import *

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [3]:
from pandas import *
# https://transition.fec.gov/pubrec/fe2016/federalelections2016_000.pdf
df = pandas.read_csv('2016_Federal_Election_Presidential_Results.csv')
# Convert dtype to int
df = df.fillna(value=0)
df.EVR = df.EVR.astype('int64')
df.EVD = df.EVD.astype('int64')
df.EVO = df.EVO.astype('int64')
df = df.set_index(df.STATE)
df = df.rename(columns={"Total Vote": "PVT"})
# Remove extraneous summary row
df = df.drop('Total')

In [4]:
# https://hudgis-hud.opendata.arcgis.com/datasets/fair-market-rents
rents_df = pandas.read_csv('2018_Fair_Market_Rents.csv')
rents_df
# Group by State to get an average rent (max by county) per state, simple string match lambda
rents_df.FMR_AREANAME = rents_df.FMR_AREANAME.apply(lambda x: str(x).split(",")[1][1:3] 
                                                    if len(str(x).split(",")) > 1 
                                                    else str(x))
rents_df = rents_df.groupby(by='FMR_AREANAME').max()

In [5]:
import math

# Calculate the derived dataframe for running the solvers
electoral_votes = []
plurality_delta = []
relocation_costs = []
for i in df.itertuples():
    # Calculate number of electoral votes per state
    electoral_votes.append(i.EVR + i.EVD + i.EVO)
    
    # Calculate Plurality Delta
    blueShare = float(i.PVD) / i.PVT
    redShare = float(i.PVR) / i.PVT
    delta = int(math.ceil((i.PVT * (min(redShare, 0.5) - blueShare)) 
                      / (1 - min(redShare, 0.5))))
    plurality_delta.append(delta)
    
    # Add relocation costs
    # State average based on HUD data 
    # Relocation cost formula = 3 months of rent for a 1 Bedroom 
    #  + average cost for movers to drive halfway across the country (https://www.movebuddha.com/moving-cost-calculator-tool/)
    #  + average one way ticket to fly halfway across the country
    cost = (rents_df.FMR_1BDR[i[0]] * 3) + 2500 + 250
    relocation_costs.append(cost)
    
d = {'electoral_votes': electoral_votes,
    'plurality_delta': plurality_delta,
    'relocation_costs': relocation_costs
}
df_derived = pandas.DataFrame(data=d, index=df.index)

In [6]:
# Set up Constraint and input arrays for PulP solver
states = list(df.index)
relocation_costs = df_derived.to_dict()['relocation_costs']
voter_gaps = df_derived.to_dict()['plurality_delta']
electoral_votes = df_derived.to_dict()['electoral_votes']

# Linear Optimization for an Electoral College Win by minimizing budget spend
def LpSolveMinBudgetForECWin(budget, 
                             states, 
                             relocation_costs, 
                             voter_gaps, 
                             electoral_votes):
    minECVotes = 270

    # Linear Optimization
    prob = LpProblem("Min Budget to Win Electoral College Problem", LpMinimize)
    state_vars = LpVariable.dicts("St", states, lowBound=0, upBound=1, cat='Integer')
    prob += lpSum([state_vars[i]*voter_gaps[i]*relocation_costs[i] if voter_gaps[i] > 0 else 0 for i in states]), "RelocationBudget"
    prob += lpSum([state_vars[i]*voter_gaps[i]*relocation_costs[i] if voter_gaps[i] > 0 else 0 for i in states]) <= budget, "MaxBudgetForRelocation"
    prob += lpSum([electoral_votes[i]*state_vars[i] for i in states]) >= minECVotes, "Min Electoral Votes To Win"
    prob.writeLP("MaxECMinBudgetModel_v2.lp")
    prob.solve()
    
    print("MINIMUM BUDGET TO WIN THE ELECTORAL COLLEGE")
    print("Status:", LpStatus[prob.status])
    evSum = 0
    targetedStates = []
    flippedStates = []
    for v in prob.variables():
        if v.varValue == 1:
            ev = electoral_votes[v.name[3:]]
            evSum += ev
        if v.varValue > 0 and df.EVR[v.name[3:]] > 0:
            targetedStates.append(v.name[3:])
        if v.varValue == 1 and df.EVR[v.name[3:]] > 0:
            flippedStates.append(v.name[3:])
    print("Total Spend = ", value(prob.objective))
    print("Total EC Votes = ", evSum)
    print("Targeted States: ", targetedStates)
    print("Flipped States: ", flippedStates)

In [7]:
# Linear Optimization for Max Electoral College Win with a given budget
def LpSolveMaxECWinForBudget(budget, states, relocation_costs, voter_gaps, electoral_votes):
    minECVotes = 270
    prob = LpProblem("Max Win Electoral College Problem", LpMaximize)
    state_vars = LpVariable.dicts("St", states, lowBound=0, upBound=1, cat='Integer')
    prob += lpSum([electoral_votes[i]*state_vars[i] for i in states]), "Electoral Votes"
    prob += lpSum([electoral_votes[i]*state_vars[i] for i in states]) >= minECVotes, "Min Electoral Votes To Win"
    prob += lpSum([state_vars[i]*voter_gaps[i]*relocation_costs[i] if voter_gaps[i] > 0 else 0 for i in states]) <= budget, "MaxBudgetForRelocation"

    prob.writeLP("MaxECWin.lp")
    prob.solve()
   
    print("MAXIMUM ELECTORAL VOTES FOR A GIVEN BUDGET")
    print("Status:", LpStatus[prob.status])
    targetedStates = []
    flippedStates = []
    for v in prob.variables():
        if v.varValue > 0 and df.EVR[v.name[3:]] > 0:
            targetedStates.append(v.name[3:])
        if v.varValue == 1 and df.EVR[v.name[3:]] > 0:
            flippedStates.append(v.name[3:])
    print("Total Electoral Votes (with a budget of %d) = %d" % (budget, value(prob.objective)))
    print("Targeted States: ", targetedStates)
    print("Flipped States: ", flippedStates)

In [8]:
def runSolvers(budget, states, relocation_costs, voter_gaps, electoral_votes):
    LpSolveMinBudgetForECWin(budget, states, relocation_costs, voter_gaps, electoral_votes)
    print("\n")
    LpSolveMaxECWinForBudget(budget, states, relocation_costs, voter_gaps, electoral_votes)

# U.S. Democratic Presidential Electoral College Relocation Solver 
## Enter a relocation budget to observe its effect on the 2016 Election
Two solvers will execute:
1. Solves the minimum budget to have spent for Democrats to have won the Electoral College
2. Solves for maximizing the Electoral College votes for a given budget

Note: Only results where **"Status: Optimal"** are valid. Otherwise, Republicans keep the win. Many caveats and assumptions described in detail in the [David Ex Machina blog post](www.davidexmachina.com).

In [9]:
budget_widget = widgets.BoundedIntText(
    value=500000000,
    min=1000000,
    max=50000000000,
    step=1000000,
    description='Budget($):',
    disabled=False
)

x = interact(runSolvers, 
         budget=budget_widget,
         states=fixed(states),
         relocation_costs=fixed(relocation_costs),
         voter_gaps=fixed(voter_gaps),
         electoral_votes=fixed(electoral_votes))


interactive(children=(BoundedIntText(value=400000000, description=u'Budget($):', max=50000000000, min=1000000,…

# Data Inputs for Balancer (Appendix)

## 2016 Presidential Election Results
- EV: Electoral Vote
- PV: Popular Vote
- R: Republican
- D: Democrat
- O: Other
- T: Total

In [10]:
df

,STATE,EVR,EVD,EVO,PVR,PVD,PVO,PVT
STATE,,,,,,,,
AL,AL,9,0,0,1318255,729547,75570,2123372
AK,AK,3,0,0,163387,116454,38767,318608
AZ,AZ,11,0,0,1252401,1161167,159597,2573165
AR,AR,6,0,0,684872,380494,65310,1130676
CA,CA,0,55,0,4483814,8753792,943998,14181604
CO,CO,0,9,0,1202484,1338870,238893,2780247
CT,CT,0,7,0,673215,897572,74133,1644920
DE,DE,0,3,0,185127,235603,23084,443814
DC,DC,0,3,0,12723,282830,15715,311268


## 2016 Derived Data for Optimization Function

In [11]:
df_derived

,electoral_votes,plurality_delta,relocation_costs
STATE,,,
AL,9,664278,5069
AK,3,85701,6845
AZ,11,177746,5771
AR,6,369688,4856
CA,55,-6244220,9515
CO,9,-240331,6662
CT,7,-379795,7301
DE,3,-86598,5375
DC,3,-281618,7112
